''' -- coding:utf-8 -- @author: Huaiyu Dong ＆ Yixing Huang @Time: 2023-10-24 @Explanition: This code aims to predict the EAB of microwave absorber structure using the Radnom forest model. '''

In [ ]:
# simple Random forest
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import time

#seed = 45
# Load your data from mydata_petals.csv
data = pd.read_csv('mydata_petals.csv')

# Extract the last column (EAB) as the target variable
EAB = data.iloc[:, -1].values

# Extract the first 10 columns as input features
input = data.iloc[:, :-1].values

# Create MinMaxScaler objects for input and EAB
scaler_in = MinMaxScaler(feature_range=(-0.5, 0.5))
scaler_out = MinMaxScaler(feature_range=(-0.99, 0.99))

# Fit and transform input data
X = scaler_in.fit_transform(input)
y = scaler_out.fit_transform(EAB.reshape(-1, 1)).ravel()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #random_state=seed
# 记录开始时间
start_time = time.time()
# Initialize and train the Random Forest model with specified hyperparameters
rf_model = RandomForestRegressor(
    n_estimators=100,  # Number of trees in the forest
    max_depth=10,  # Maximum depth of each tree (adjust as needed)
    min_samples_split=2,  # Minimum number of samples required to split an internal node
    min_samples_leaf=1  # Minimum number of samples required to be at a leaf node
) #    ,random_state=seed

rf_model.fit(X_train, y_train)
# 记录结束时间
end_time = time.time()
# 计算执行时间
execution_time = end_time - start_time
print(f"Code execution time: {execution_time:.2f} seconds")

# Predict with the trained model
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Transform the predictions back to the original scale
y_train_pred = scaler_out.inverse_transform(y_train_pred.reshape(-1, 1)).ravel()
y_test_pred = scaler_out.inverse_transform(y_test_pred.reshape(-1, 1)).ravel()
y_train_true = scaler_out.inverse_transform(y_train.reshape(-1, 1)).ravel()
y_test_true = scaler_out.inverse_transform(y_test.reshape(-1, 1)).ravel()

# Evaluate the model
train_r2 = r2_score(y_train_true, y_train_pred)
test_r2 = r2_score(y_test_true, y_test_pred)
train_mse = mean_squared_error(y_train_true, y_train_pred)
test_mse = mean_squared_error(y_test_true, y_test_pred)

print(f"Training R-squared: {train_r2:.3f}")
print(f"Testing R-squared: {test_r2:.3f}")
print(f"Training Mean Squared Error: {train_mse:.3f}")
print(f"Testing Mean Squared Error: {test_mse:.3f}")

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(y_train_true, y_train_pred, label='Training Data')
plt.scatter(y_test_true, y_test_pred, label='Testing Data')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.legend()
plt.title('True vs. Predicted Values')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_train_pred, y_train_pred - y_train_true, label='Training Data')
plt.scatter(y_test_pred, y_test_pred - y_test_true, label='Testing Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='k', linestyle='--')
plt.legend()
plt.title('Residual Plot')
plt.grid(True)
plt.show()

In [ ]:
# Additional code to plot correlation
actual_train = y_train_true
predicted_train = y_train_pred
actual_test = y_test_true
predicted_test = y_test_pred

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Scatter plot of actual values and regression line for training set
axes[0].scatter(actual_train, actual_train, color='blue', label='Actual', marker='o', s=10)
axes[0].scatter(actual_train, predicted_train, color='red', label='Predicted', marker='x', s=10)
#axes[0].set_xlim(-1.2, 1.2)
#axes[0].set_ylim(-1.2, 1.2)
correlation_train = np.corrcoef(actual_train, predicted_train, rowvar=False)[0, 1]
axes[0].set_title(f'Training Set\nCorrelation: {correlation_train:.3f}')
axes[0].legend()


# Scatter plot of actual values and regression line for test set
axes[1].scatter(actual_test, actual_test, color='blue', label='Actual', marker='o', s=10)
axes[1].scatter(actual_test, predicted_test, color='red', label='Predicted', marker='x', s=10)
#axes[1].set_xlim(-1.2, 1.2)
#axes[1].set_ylim(-1.2, 1.2)
correlation_test = np.corrcoef(actual_test, predicted_test, rowvar=False)[0, 1]
axes[1].set_title(f'Test Set\nCorrelation: {correlation_test:.3f}')
axes[1].legend()

plt.tight_layout()
plt.show()

#集成AdaBoost的随机森林
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

seed = 45

# Load your data from mydata_petals.csv
data = pd.read_csv('mydata_petals.csv')

# Extract the last column (EAB) as the target variable
EAB = data.iloc[:, -1].values

# Extract the first 10 columns as input features
input = data.iloc[:, :-1].values

# Create MinMaxScaler objects for input and EAB
scaler_in = MinMaxScaler(feature_range=(-0.5, 0.5))
scaler_out = MinMaxScaler(feature_range=(-0.99, 0.99))

# Fit and transform input data
X = scaler_in.fit_transform(input)
y = scaler_out.fit_transform(EAB.reshape(-1, 1)).ravel()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Initialize and train the AdaBoost model with Random Forest as base estimator
base_rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=seed
)

ada_model = AdaBoostRegressor(
    base_rf_model,
    n_estimators=50,  # Number of boosting stages
    random_state=seed
)

ada_model.fit(X_train, y_train)

# Predict with the trained model
y_train_pred = ada_model.predict(X_train)
y_test_pred = ada_model.predict(X_test)

# Transform the predictions back to the original scale
y_train_pred = scaler_out.inverse_transform(y_train_pred.reshape(-1, 1)).ravel()
y_test_pred = scaler_out.inverse_transform(y_test_pred.reshape(-1, 1)).ravel()
y_train_true = scaler_out.inverse_transform(y_train.reshape(-1, 1)).ravel()
y_test_true = scaler_out.inverse_transform(y_test.reshape(-1, 1)).ravel()

# Evaluate the model
train_r2 = r2_score(y_train_true, y_train_pred)
test_r2 = r2_score(y_test_true, y_test_pred)
train_mse = mean_squared_error(y_train_true, y_train_pred)
test_mse = mean_squared_error(y_test_true, y_test_pred)

print(f"Training R-squared: {train_r2:.4f}")
print(f"Testing R-squared: {test_r2:.4f}")
print(f"Training Mean Squared Error: {train_mse:.4f}")
print(f"Testing Mean Squared Error: {test_mse:.4f}")

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(y_train_true, y_train_pred, label='Training Data')
plt.scatter(y_test_true, y_test_pred, label='Testing Data')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.legend()
plt.title('True vs. Predicted Values')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_train_pred, y_train_pred - y_train_true, label='Training Data')
plt.scatter(y_test_pred, y_test_pred - y_test_true, label='Testing Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='k', linestyle='--')
plt.legend()
plt.title('Residual Plot')
plt.grid(True)
plt.show()

#集成梯度提升（Gradient Boosting）的random forest model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

seed = 45

# Load your data from mydata_petals.csv
data = pd.read_csv('mydata_petals.csv')

# Extract the last column (EAB) as the target variable
EAB = data.iloc[:, -1].values

# Extract the first 10 columns as input features
input = data.iloc[:, :-1].values

# Create MinMaxScaler objects for input and EAB
scaler_in = MinMaxScaler(feature_range=(-0.5, 0.5))
scaler_out = MinMaxScaler(feature_range=(-0.99, 0.99))

# Fit and transform input data
X = scaler_in.fit_transform(input)
y = scaler_out.fit_transform(EAB.reshape(-1, 1)).ravel()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Initialize and train the Random Forest model with specified hyperparameters
rf_model = RandomForestRegressor(
    n_estimators=100,  # Number of trees in the forest
    max_depth=10,  # Maximum depth of each tree (adjust as needed)
    min_samples_split=2,  # Minimum number of samples required to split an internal node
    min_samples_leaf=1,  # Minimum number of samples required to be at a leaf node
    random_state=seed
) 

rf_model.fit(X_train, y_train)

# Predict with the trained Random Forest model
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

# Initialize and train the Gradient Boosting model with specified hyperparameters
gb_model = GradientBoostingRegressor(
    n_estimators=100,  # Number of boosting stages (trees)
    learning_rate=0.1,  # Step size shrinking to prevent overfitting
    max_depth=4,  # Maximum depth of each tree (adjust as needed)
    random_state=seed
)

gb_model.fit(X_train, y_train)

# Predict with the trained Gradient Boosting model
y_train_pred_gb = gb_model.predict(X_train)
y_test_pred_gb = gb_model.predict(X_test)

# Transform the predictions back to the original scale
y_train_pred_rf = scaler_out.inverse_transform(y_train_pred_rf.reshape(-1, 1)).ravel()
y_test_pred_rf = scaler_out.inverse_transform(y_test_pred_rf.reshape(-1, 1)).ravel()
y_train_pred_gb = scaler_out.inverse_transform(y_train_pred_gb.reshape(-1, 1)).ravel()
y_test_pred_gb = scaler_out.inverse_transform(y_test_pred_gb.reshape(-1, 1)).ravel()
y_train_true = scaler_out.inverse_transform(y_train.reshape(-1, 1)).ravel()
y_test_true = scaler_out.inverse_transform(y_test.reshape(-1, 1)).ravel()

# Evaluate the Random Forest model
train_r2_rf = r2_score(y_train_true, y_train_pred_rf)
test_r2_rf = r2_score(y_test_true, y_test_pred_rf)
train_mse_rf = mean_squared_error(y_train_true, y_train_pred_rf)
test_mse_rf = mean_squared_error(y_test_true, y_test_pred_rf)

# Evaluate the Gradient Boosting model
train_r2_gb = r2_score(y_train_true, y_train_pred_gb)
test_r2_gb = r2_score(y_test_true, y_test_pred_gb)
train_mse_gb = mean_squared_error(y_train_true, y_train_pred_gb)
test_mse_gb = mean_squared_error(y_test_true, y_test_pred_gb)

print("Random Forest Model:")
print(f"Training R-squared: {train_r2_rf:.4f}")
print(f"Testing R-squared: {test_r2_rf:.4f}")
print(f"Training Mean Squared Error: {train_mse_rf:.4f}")
print(f"Testing Mean Squared Error: {test_mse_rf:.4f}")

print("Gradient Boosting Model:")
print(f"Training R-squared: {train_r2_gb:.4f}")
print(f"Testing R-squared: {test_r2_gb:.4f}")
print(f"Training Mean Squared Error: {train_mse_gb:.4f}")
print(f"Testing Mean Squared Error: {test_mse_gb:.4f}")

# Plot the results for Random Forest
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(y_train_true, y_train_pred_rf, label='Training Data (RF)')
plt.scatter(y_test_true, y_test_pred_rf, label='Testing Data (RF)')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.legend()
plt.title('True vs. Predicted Values (RF)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_train_pred_rf, y_train_pred_rf - y_train_true, label='Training Data (RF)')
plt.scatter(y_test_pred_rf, y_test_pred_rf - y_test_true, label='Testing Data (RF)')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='k', linestyle='--')
plt.legend()
plt.title('Residual Plot (RF)')
plt.grid(True)

# Plot the results for Gradient Boosting
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(y_train_true, y_train_pred_gb, label='Training Data (GB)')
plt.scatter(y_test_true, y_test_pred_gb, label='Testing Data (GB)')
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.legend()
plt.title('True vs. Predicted Values (GB)')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_train_pred_gb, y_train_pred_gb - y_train_true, label='Training Data (GB)')
plt.scatter(y_test_pred_gb, y_test_pred_gb - y_test_true, label='Testing Data (GB)')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='k', linestyle='--')
plt.legend()
plt.title('Residual Plot (GB)')
plt.grid(True)

plt.show()